In [2]:
import io
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots

## Uploading data to colab


In [3]:
from google.colab import files
 
uploaded = files.upload()

Saving data-1646226583655.csv to data-1646226583655 (3).csv


## Loading data

In [17]:
# Read data from colab
data = pd.read_csv(io.BytesIO(uploaded['data-1646226583655.csv']))
# Read local data
# data = pd.read_csv('file-path.csv')

# removing timezone
data.timeframe = data.timeframe.apply(lambda x: pd.to_datetime(x[:-6]))
data = data.sort_values(["timeframe"])

starting_date = pd.Timestamp(data[:1].timeframe.values[0])
end_date = pd.Timestamp(data[-1:].timeframe.values[0])

print(data.describe())

print()
print("Starting date: ", starting_date)
print("End date: ", end_date)

       pool_id       low_1      high_1  ...     close_1      open_2     close_2
count    422.0  422.000000  422.000000  ...  422.000000  422.000000  422.000000
mean       3.0    0.343604    0.351747  ...    0.347288    3.128530    3.133793
std        0.0    0.095913    0.108910  ...    0.103338    1.164669    1.177859
min        3.0    0.099684    0.101602  ...    0.101219    1.091136    1.088941
25%        3.0    0.333334    0.333334  ...    0.333334    2.999977    2.999976
50%        3.0    0.333335    0.333335  ...    0.333335    2.999987    2.999987
75%        3.0    0.333336    0.333336  ...    0.333336    2.999996    2.999996
max        3.0    0.900824    0.918877  ...    0.918324    9.844558    9.879541

[8 rows x 9 columns]

Starting date:  2022-03-01 21:51:00
End date:  2022-03-02 13:09:00


In [18]:
data

,timeframe,pool_id,low_1,high_1,low_2,high_2,open_1,close_1,open_2,close_2
421,2022-03-01 21:51:00,3,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
420,2022-03-01 21:53:00,3,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
419,2022-03-01 21:54:00,3,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
418,2022-03-01 21:56:00,3,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
417,2022-03-01 21:57:00,3,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...
4,2022-03-02 13:04:00,3,0.205964,0.226469,4.415606,4.855214,0.205964,0.221259,4.855214,4.519592
3,2022-03-02 13:05:00,3,0.226506,0.289584,3.453235,4.414898,0.226506,0.278781,4.414898,3.587045
2,2022-03-02 13:06:00,3,0.289584,0.295353,3.385775,3.453233,0.289584,0.295322,3.453233,3.386135
1,2022-03-02 13:07:00,3,0.299237,0.365722,2.734317,3.341830,0.299237,0.299237,3.341830,3.341830


## Aggregating missing timeframes

In [19]:
explicit_pool = data[data['pool_id'] == 3]
explicit_pool = explicit_pool.drop(columns=['pool_id'])

def correct_line(rep, curr):
  return lambda line: line[rep] if line['missing_data'] else line[curr]

new_data = pd.DataFrame(pd.date_range(start=starting_date, end=end_date, freq='min'), columns=["timeframe"])
new_data = new_data.join(explicit_pool.set_index('timeframe'), on='timeframe')
new_data = new_data.sort_values(['timeframe'])
new_data['missing_data'] = new_data['close_1'].apply(pd.isnull)
new_data['close_1'] = new_data['close_1'].fillna(method='ffill')
new_data['close_2'] = new_data['close_2'].fillna(method='ffill')

new_data['low_1'] = new_data.apply(correct_line('close_1', 'low_1'), axis=1)
new_data['high_1'] = new_data.apply(correct_line('close_1', 'high_1'), axis=1)
new_data['open_1'] = new_data.apply(correct_line('close_1', 'open_1'), axis=1)

new_data['low_2'] = new_data.apply(correct_line('close_2', 'low_2'), axis=1)
new_data['high_2'] = new_data.apply(correct_line('close_2', 'high_2'), axis=1)
new_data['open_2'] = new_data.apply(correct_line('close_2', 'open_2'), axis=1)

new_data = new_data.drop(columns=['missing_data'])
new_data

,timeframe,low_1,high_1,low_2,high_2,open_1,close_1,open_2,close_2
0,2022-03-01 21:51:00,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
1,2022-03-01 21:52:00,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
2,2022-03-01 21:53:00,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
3,2022-03-01 21:54:00,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
4,2022-03-01 21:55:00,0.333333,0.333333,3.000000,3.000000,0.333333,0.333333,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...
914,2022-03-02 13:05:00,0.226506,0.289584,3.453235,4.414898,0.226506,0.278781,4.414898,3.587045
915,2022-03-02 13:06:00,0.289584,0.295353,3.385775,3.453233,0.289584,0.295322,3.453233,3.386135
916,2022-03-02 13:07:00,0.299237,0.365722,2.734317,3.341830,0.299237,0.299237,3.341830,3.341830
917,2022-03-02 13:08:00,0.299237,0.299237,3.341830,3.341830,0.299237,0.299237,3.341830,3.341830


In [20]:

df = new_data.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace( 
  go.Candlestick(
    x=df['timeframe'],
    open=df['open_2'],
    high=df['high_2'],
    low=df['low_2'],
    close=df['close_2'],
    name="Ratio 2"
  ),
  secondary_y=False,
)
fig.add_trace(
  go.Candlestick(
    x=df['timeframe'],
    open=df['open_1'],
    high=df['high_1'],
    low=df['low_1'],
    close=df['close_1'],
    name="Ratio 1",
    increasing_line_color= 'cyan',
    decreasing_line_color= 'gray'
    
  ),
  secondary_y=True,
)
fig.show()

In [16]:
df = new_data.copy()

fig = go.Figure(
    data=[
      go.Candlestick(
        x=df['timeframe'],
        open=df['open_2'],
        high=df['high_2'],
        low=df['low_2'],
        close=df['close_2']
      )
    ]
)

fig.show()